<a href="https://colab.research.google.com/github/monteroanibal/diplomado_GEOIA_IGAC2025/blob/main/unidad_1_6_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook (III)

---

# 1. Cargue de librerías

Para generar los mapas interactivos.

In [15]:
import plotly.express as px
import plotly.graph_objects as go

Para pasar los archivos planos de tablas a GeoDataFrame.

In [16]:
import geopandas as gpd
import pandas as pd

# 2. Lectura y manipulación de datos

Se descargan las capas que están publicadas en un Github y se descomprimen en la sesión actual.

In [17]:
%%capture
!wget https://github.com/monteroanibal/diplomado_GEOIA_IGAC2025/raw/refs/heads/main/Insumos/Unidad_1_6_Datos/shapefiles.zip
!unzip -o /content/shapefiles.zip
!rm /content/shapefiles.zip

In [18]:
%%capture
!wget https://github.com/monteroanibal/diplomado_GEOIA_IGAC2025/raw/refs/heads/main/Insumos/Unidad_1_6_Datos/libros_Excel.zip
!unzip -o /content/libros_Excel.zip
!rm /content/libros_Excel.zip

Se importan las capas descargadas.

In [19]:
sismos = pd.read_excel("/content/libros_Excel/SGC-sismicidad_historica.xlsx")
nacims = pd.read_excel("/content/libros_Excel/DANE-EEVV-Nacimientos2023.xlsx", converters={'DPTO_CODIGO':str})
mgndpt = gpd.read_file("/content/shapefiles/MGN2023_DPTO_POLITICO/MGN_ADM_DPTO_POLITICO.shp")

Se edita el campo de código para que utilice el cero a la izquierda de algunos de los departamentos.


In [20]:
nacims.loc[nacims.DPTO_CODIGO == '3', 'DPTO_CODIGO'] = '03'
nacims.loc[nacims.DPTO_CODIGO == '5', 'DPTO_CODIGO'] = '05'
nacims.loc[nacims.DPTO_CODIGO == '8', 'DPTO_CODIGO'] = '08'

Se reproyecta la capa de información de MGN para que funcione correctamente la librería _plotly_.

In [21]:
mgndpt = mgndpt.to_crs(epsg=4326)

Se genera el GeoDataFrame a partir de la tabla de datos importada desde el libro Excel, especificando las coordenadas geográficas como la geometría de punto del mismo.

In [22]:
sismos = gpd.GeoDataFrame( sismos, geometry=gpd.points_from_xy(sismos.Longitud, sismos.Latitud), crs="EPSG:4326")
sismos['Profundidad'] = sismos['Profundidad']*(-1)

Por medio del código DIVIPOLA, se cruza los datos geográficos de los departamentos con los de sus respectivas cantidades de nacimientos.

In [23]:
nacims = nacims.loc[nacims['Area'] == 'Total']
nacims = pd.merge(nacims, mgndpt, left_on='DPTO_CODIGO', right_on='dpto_ccdgo', ).set_geometry("geometry").set_index("dpto_cnmbr")

# 3. Despliegue de scatterplots con _plotly_: sismicidad histórica en Colombia

A continuación, se despliega un diagrama de dispersión a partir de los datos de sismicidad. Se resalta la capacidad de _plotly_ de hacer explorqación interactiva en los gráficos, entre los que está la rotación, el zoom y el despliegue del _tooltip_. Como se observa en el código, el gráfico es recuperable en archivo extensión _.html_.

In [ ]:
fig = px.scatter_3d(sismos, x='Magnitud', y='IntensidadMax', z='Profundidad', color='Magnitud', hover_data=['Area_epicentral','Fecha','Hora_local'])
fig.update_layout(
    title = 'Sismiscidad histórica en Colombia (SGC)',
    height=500, width=700, coloraxis_showscale=False
)
fig.show()
fig.write_html("scatterplot_sismicidad.html")

A continuación, se muestra un ejemplo de generación de diagrama de dispersión por ubicación geográfica. Además de las funcionalidades del anterior gráfico, estas incluyen el desplazamiento.

In [25]:
sismos['Descripcion'] = sismos['Area_epicentral'] + '<br>Fecha y hora: ' + sismos['Fecha'].astype(str) + ', ' + sismos['Hora_local'].astype(str) + '<br>Magnitud: ' + sismos['Magnitud'].astype(str) + '<br>Profundidad: ' + sismos['Profundidad'].astype(str) + '<br>Intensidad máxima: ' + sismos['IntensidadMax'].astype(str)

In [ ]:
fig = go.Figure( data=go.Scattergeo(
  lon = sismos['Longitud'],
  lat = sismos['Latitud'],
  text = sismos['Descripcion'],
  mode = 'markers',
  marker = dict(
    size = 8,
    opacity = 0.8,
    reversescale = True,
    autocolorscale = False,
    symbol = 'square',
    line = dict(
      width=1,
      color='rgba(102, 102, 102)'
    ),
    colorscale = 'Plasma',
    cmin = 0,
    color = sismos['Magnitud'],
    cmax = sismos['Magnitud'].max(),
    colorbar=dict( title=dict( text="Magnitud" ) )
) ) )

fig.update_geos(projection_type="orthographic")

fig.update_layout(
    title = 'Sismiscidad histórica en Colombia (SGC)',
    height=500, width=700
)

fig.show()
fig.write_html("scatterplot_geo_sismicidad.html")

# 4. Despliegue de cloropletas con _plotly_: Natalidad en Colombia, 2023

A continuación, se muestra un ejemplo de generación de cloropleta, incluyendo todas las funcionalidades interactivas antes descritas.

In [ ]:
fig = px.choropleth(
  nacims, geojson=nacims.geometry, locations=nacims.index, color="Total",
  hover_data=['Peso_2000_2499','Peso_2500_2999','Peso_3000_3499','Peso_3500_3999','Peso_4000_y_mas','Sin_informacion']
)

fig.update_geos(fitbounds="locations")
fig.update_layout(
  title = 'Natalidad en Colombia, 2023 (DANE-EEVV)',
  height=500, width=700
)
fig.show()
fig.write_html("/content/cloropleta_plotly_natalidad.html")

**<center>Un aviso**</center>

_plotly_ es una herramienta inicial para el desarrollo web que da paso al _framework_ llamado _dash_, con el que se puede incluir la funcionalidad de filtrado y de _tooltip_ cruzado, similar a los despliegues de sowtware para Inteligencia de Negocios, como _PowerBI_ y Tableau.